The code below results from following the Youtube video:
<i>Convolutional Neural Network from Scratch | Mathematics & Python Code</i> by The Independent Code https://www.youtube.com/watch?v=Lakz2MoHy6o

In [1]:
import numpy as np
from scipy import signal
from keras.datasets import mnist
from keras.utils import np_utils

### Layer Class

In [2]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
        
    def forward(self, input):
        pass
        
    def backward(self, output_gradient, learning_rate):
        pass

### Dense Layer Class

In [3]:
class Dense(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size)
        self.bias = np.random.randn(output_size, 1)
    
    def forward(self, input):
        self.input = input
        return np.dot(self.weights, self.input) + self.bias
    
    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(output_gradient, self.input.T)
        self.weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * output_gradient
        return np.dot(self.weights.T, output_gradient)

### Activation Layer Class

In [4]:
class Activation(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime
        
    def forward(self, input):
        self.input = input
        return self.activation(self.input)
        
    def backward(self, output_gradient, learning_rate):
        return np.multiply(output_gradient, self.activation_prime(self.input))
    

### Convolutional Layer Class

In [5]:
class Convolutional(Layer):
    def __init__(self, input_shape, kernel_size, depth):
        input_depth, input_height, input_width = input_shape
        self.depth = depth
        self.input_shape = input_shape
        self.input_depth = input_depth
        self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1)
        self.kernels_shape = (depth, input_depth, kernel_size, kernel_size)
        self.kernels = np.random.randn(*self.kernels_shape)
        self.biases = np.random.randn(*self.output_shape)
    
    def forward(self, input):
        self.input = input
        self.output = np.copy(self.biases)
        for i in range(self.depth):
            for j in range(self.input_depth):
                self.output[i] += signal.correlate2d(self.input[j], self.kernels[i,j], "valid")
        return self.output
    
    def backward(self, output_gradient, learning_rate):
        kernels_gradient = np.zeros(self.kernels_shape)
        input_gradient = np.zeros(self.input_shape)
        for i in range(self.depth):
            for j in range(self.input_depth):
                kernels_gradient[i, j] = signal.correlate2d(self.input[j], output_gradient[i], "valid")
                input_gradient[j] += signal.convolve2d(output_gradient[i], self.kernels[i, j], "full")
                
        self.kernels -= learning_rate * kernels_gradient
        self.biases -= learning_rate * output_gradient
        return input_gradient

### Reshape Layer Class

In [6]:
class Reshape(Layer):
    def __init__(self, input_shape, output_shape):
        self.input_shape = input_shape
        self.output_shape = output_shape
    
    def forward(self, input):
        return np.reshape(input, self.output_shape)
    
    def backward(self, output_gradient, learning_rate):
        return np.reshape(output_gradient, self.input_shape)

### Binary Cross Entropy and its derivative

In [7]:
def binary_cross_entropy(y_true, y_pred):
    return np.mean(-y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred))

def binary_cross_entropy_prime(y_true, y_pred):
    return ((1 - y_true) / (1 - y_pred) - y_true / y_pred) / np.size(y_true)

### Sigmoid Activation Function

In [8]:
class Sigmoid(Activation):
    def __init__(self):
        
        def sigmoid(x):
            return 1 / (1 + np.exp(-x))
        
        def sigmoid_prime(x):
            s = sigmoid(x)
            return s * (1 - s)
        
        super().__init__(sigmoid, sigmoid_prime)
        

### Building the Convolutional Neural Net

In [9]:
#get the 0s and 1s only
def preprocess_data(x, y, limit):
    zero_index = np.where(y == 0)[0][:limit]
    one_index = np.where(y == 1)[0][:limit]
    all_indices = np.hstack((zero_index, one_index))
    all_indices = np.random.permutation(all_indices)
    x, y = x[all_indices], y[all_indices]
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype("float32") / 255
    y = np_utils.to_categorical(y)
    y = y.reshape(len(y), 2, 1)
    return x, y

#load mnist from server, limit 100 images per class sincewe're not training on GPU
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = preprocess_data(x_train, y_train, 100)
x_test, y_test = preprocess_data(x_test, y_test, 100)

#create the network
network = [
    Convolutional((1, 28, 28), 3, 5),
    Sigmoid(),
    Reshape((5, 26, 26), (5 * 26 * 26, 1)),
    Dense(5 * 26 * 26, 100),
    Sigmoid(),
    Dense(100, 2),
    Sigmoid()
]

epochs = 20
learning_rate = 0.1

#train
for e in range(epochs):
    error = 0
    for x, y in zip(x_train, y_train):
        #forward
        output = x
        for layer in network:
            output = layer.forward(output)
        #error
        error += binary_cross_entropy(y, output)
        
        #backward
        grad = binary_cross_entropy_prime(y, output)
        for layer in reversed(network):
            grad = layer.backward(grad, learning_rate)
            
    error /= len(x_train)
    print(f"{e + 1}/{epochs}, error={error}")
          
#test
for x, y in zip(x_test, y_test):
    output = x
    for layer in network:
        output = layer.forward(output)
    print(f"pred= {np.argmax(output)}, true= {np.argmax(y)}")

1/20, error=0.5814175548908144
2/20, error=0.0882168646854503
3/20, error=0.052302415507725845
4/20, error=0.019908916761302846
5/20, error=0.012524922825159488
6/20, error=0.008911813988264888
7/20, error=0.006780728471651548
8/20, error=0.005569633984171646
9/20, error=0.004692452554342192
10/20, error=0.004137382903080813
11/20, error=0.0037311227788435148
12/20, error=0.003408452386426053
13/20, error=0.0031408352338877254
14/20, error=0.0029140219037604503
15/20, error=0.0027188985978170745
16/20, error=0.002549029081489022
17/20, error=0.0023996670007102633
18/20, error=0.002267215731098402
19/20, error=0.002148893766803135
20/20, error=0.0020425135747029516
pred= 1, true= 1
pred= 0, true= 0
pred= 1, true= 1
pred= 0, true= 0
pred= 0, true= 0
pred= 1, true= 1
pred= 1, true= 1
pred= 1, true= 1
pred= 0, true= 0
pred= 1, true= 1
pred= 0, true= 0
pred= 1, true= 0
pred= 0, true= 0
pred= 1, true= 0
pred= 1, true= 1
pred= 0, true= 0
pred= 1, true= 1
pred= 1, true= 1
pred= 1, true= 1
pred